# Churn Prediction

#### use binary clasification to represent negative churn 0 and positive churn 1

In [15]:
import pandas as pd
import numpy as np

# Data - Prep

In [16]:
import urllib.request

url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
filename = "course_lead_scoring.csv"
urllib.request.urlretrieve(url, filename)

df = pd.read_csv(filename)
#df.head()
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


#### Check if the missing values are presented in the features.
### If there are missing values:
##### 1. For caterogiral features, replace them with 'NA'
##### 2. For numerical features, replace with with 0.0

In [17]:
#### Check if the missing values are presented in the features:

# Create a summary DataFrame with column name, dtype, and null counts
summary_df = pd.DataFrame({
    "Column": df.columns,
    "Dtype": df.dtypes.values,
    "Null_Count": df.isnull().sum().values
})

print(summary_df)

                     Column    Dtype  Null_Count
0               lead_source   object         128
1                  industry   object         134
2  number_of_courses_viewed    int64           0
3             annual_income  float64         181
4         employment_status   object         100
5                  location   object          63
6         interaction_count    int64           0
7                lead_score  float64           0
8                 converted    int64           0


In [18]:
#### If there are missing values:
#### 1. For caterogiral features, replace them with 'NA'

df[['lead_source', 'industry', 'employment_status', 'location']] = df[['lead_source', 'industry', 'employment_status', 'location']].fillna('NA')

#### 2. For numerical features, replace with with 0.0

df['annual_income'] = df['annual_income'].fillna(0)

check_summary_df = pd.DataFrame({
    "Column": df.columns,
    "Dtype": df.dtypes.values,
    "Null_Count": df.isnull().sum().values
})

print(check_summary_df)

                     Column    Dtype  Null_Count
0               lead_source   object           0
1                  industry   object           0
2  number_of_courses_viewed    int64           0
3             annual_income  float64           0
4         employment_status   object           0
5                  location   object           0
6         interaction_count    int64           0
7                lead_score  float64           0
8                 converted    int64           0


In [19]:
# if churn col isn't in binary, make it so
#df.churn = (df.churn == 'yes').astype(int).head()

# Question 1
#### What is the most frequent observation (mode) for the column industry?

In [20]:
df['industry'].mode()
#df['industry'].value_counts().get('retail', 0)

0    retail
Name: industry, dtype: object

# Question 2
#### Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.
#### 
What are the two features that have the biggest correlation?

# Split the data
Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value y is not in your dataframe.

In [21]:
from sklearn.model_selection import train_test_split

In [28]:
df_full_train, df_test = train_test_split(df, train_size=0.8, test_size=0.2, random_state=42)

len(df_full_train) #1169
len(df_test) #293

293

In [39]:
df_train, df_val = train_test_split(df_full_train, train_size=0.75, test_size=0.25, random_state=42)
len(df_train) #876
len(df_val) #293
len(df_test) # 293

876

In [40]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

df_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03,0
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77,1
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59,1
3,NA,technology,1,74956.0,employed,europe,3,0.34,1
4,organic_search,retail,3,59335.0,student,australia,1,0.98,1


In [41]:
y_train = df_train.converted.values
y_test = df_test.converted.values
y_val = df_val.converted.values

In [46]:
# del df_train['converted']
# del df_test['converted']
# del df_val['converted']
df_train.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
3,NA,technology,1,74956.0,employed,europe,3,0.34
4,organic_search,retail,3,59335.0,student,australia,1,0.98


# Question 3
Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2)
Which of these variables has the biggest mutual information score?.

# Question 4
Now let's train a logistic regression.
Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
Fit the model on the training dataset.
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
What accuracy did you get?

# Question 5
Let's find the least useful feature using the feature elimination technique.
Train a model using the same features and parameters as in Q4 (without rounding).
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
Which of following feature has the smallest difference?

# Question 6
Now let's train a regularized logistic regression.
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
Train models using all the features as in Q4.
Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
Which of these C leads to the best accuracy on the validation set? 